In [1]:
import pandas as pd

# RAIS

## Estabelecimentos


TODO

- Pipeline

    - Leitura
    - Tratamento
    - Merging
    - Load

ano, uf, meso, micro, muni

In [2]:
def read_data(caminho, ano):
    try: # txt file
        colunas = ['CNAE 2.0 Classe', 'Município']
        df_estb = pd.read_csv(caminho + 'ESTB' + ano + '.txt', sep=';', encoding='latin1', usecols=colunas)
        df_estb['ano'] = ano
        df_estb.rename(columns={'Município': 'id_municipio', 'CNAE 2.0 Classe': 'cnae_2'}, inplace=True)	
        df_estb = df_estb[['ano', 'id_municipio', 'cnae_2']]
    except: # csv file
        colunas = ['ano', 'id_municipio', 'cnae_2']
        df_estb = pd.read_csv(caminho + 'ESTB' + ano + '.csv', usecols=colunas)
        
        df_estb.dropna(inplace=True)
        df_estb['id_municipio'] = df_estb['id_municipio'].astype('int')
        
        df_estb['id_municipio'] = df_estb['id_municipio'].apply(lambda x: int(x/10)) # remove 7'th dígito

    return df_estb

In [3]:
def transform_data(df):
    '''Normaliza o tipo de cada coluna e obtem a cnae_secao de cnae_2'''
    
    df['ano'] = df['ano'].astype('int16')
    
    df['id_municipio'] = df['id_municipio'].astype('int')
        
    df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))
    df['cnae_secao'] = df['cnae_2'].apply(lambda x: x[:2])
    df['cnae_secao'] = df['cnae_secao'].astype('int')
    df.drop(columns=['cnae_2'], inplace=True)
    return df

In [4]:
def merge_municipios(df, caminho):
    '''Faz o merge do id_municipio com os dados do dicionário'''
    
    colunas_muni = ['id_municipio_6', 'nome', 'nome_microrregiao', 'nome_mesorregiao', 'sigla_uf', 'nome_uf', 'nome_regiao']
    dicionario_muni = pd.read_csv(caminho, sep=',', usecols=colunas_muni)
    dicionario_muni.rename(columns={'id_municipio_6': 'id_municipio', 'nome': 'municipio'}, inplace=True)
    
    df = pd.merge(df, dicionario_muni, on='id_municipio', how='left')
    df.drop(columns=['id_municipio'], inplace=True)
    return df

In [5]:
def merge_cnae(df, caminho):
    
    colunas_tmp = ['divisao','descricao_secao']
    df_cnae = pd.read_csv(caminho, usecols=colunas_tmp)
    df_cnae.rename(columns={'divisao': 'cnae_secao'}, inplace=True)
    df_cnae.drop_duplicates(inplace=True)
    
    df = pd.merge(df, df_cnae, on='cnae_secao', how='left')
    df.drop(columns=['cnae_secao'], inplace=True)
    return df

In [6]:
def calc_ql(df, loc='meso'):
    if loc == 'meso':
        numerador = df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'nome_mesorregiao', 'descricao_secao', 'quociente_localizacao']
    elif loc == 'micro':
        numerador = df.groupby(['ano', 'sigla_uf', 'nome_microrregiao', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'nome_microrregiao']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql.columns = ['ano', 'sigla_uf', 'nome_microrregiao', 'descricao_secao', 'quociente_localizacao']
        ql = ql.reset_index()
    
    ql['quociente_localizacao'] = ql['quociente_localizacao'].map(lambda x: float(f"{x:.2f}"))


    return ql


In [7]:
def etl_pipeline(caminhos, ano_ini, ano_fim, loc):
    dfs_ql = []
    for ano in range(ano_ini, ano_fim+1):
        print('---', ano, '---')
        df = read_data(caminhos['rais'], str(ano))
        df = transform_data(df)
        df = merge_municipios(df, caminhos['municipios'])
        df = merge_cnae(df, caminhos['cnae'])
        
        dfs_ql.append(calc_ql(df, loc))
    df = pd.concat(dfs_ql)
    return df

In [8]:
caminho = "D:/dados/rais/estabelecimentos/"
caminho_dict_muni = "D:/dados/rais/dicionarios/dicionario_municipios.csv"
caminho_dict_cnae = "D:/dados/rais/dicionarios/dicionario_cnae_2.csv"
ano_ini = 2021
ano_fim = 2021
caminhos = {'rais': caminho, 
            'municipios': caminho_dict_muni, 
            'cnae': caminho_dict_cnae}

df = etl_pipeline(caminhos, ano_ini, ano_fim, 'meso')
df

--- 2021 ---


C:\Users\ramos\AppData\Local\Temp\ipykernel_20772\2426591515.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1113.0' '1113.0' '1113.0' ... '96033.0' '96092.0' '96092.0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))


,ano,sigla_uf,nome_mesorregiao,descricao_secao,quociente_localizacao
0,2021,AC,Vale do Acre,"Administração Pública, Defesa E Seguridade Social",1.07
1,2021,AC,Vale do Juruá,"Administração Pública, Defesa E Seguridade Social",0.74
2,2021,AC,Vale do Acre,Alojamento E Alimentação,1.09
3,2021,AC,Vale do Juruá,Alojamento E Alimentação,0.70
4,2021,AC,Vale do Acre,"Artes, Cultura, Esporte E Recreação",1.09
...,...,...,...,...,...
2551,2021,TO,Oriental do Tocantins,Serviços Domésticos,1.42
2552,2021,TO,Ocidental do Tocantins,"Transporte, Armazenagem E Correio",1.20
2553,2021,TO,Oriental do Tocantins,"Transporte, Armazenagem E Correio",0.74
2554,2021,TO,Ocidental do Tocantins,"Água, Esgoto, Atividades De Gestão De Resíduos...",0.90


In [72]:
df.groupby('ano').size()

ano
2007    2830
2008    2825
2009    2822
2010    2810
2011    2813
2012    2807
2013    2821
2014    2812
2015    2812
2016    2811
2017    2806
2018    2802
2019    2788
2020    2772
2021    2556
2022    2735
2023    2734
2024    2731
dtype: int64

In [73]:
df.query('nome_mesorregiao == "Norte de Minas"')

,ano,sigla_uf,nome_mesorregiao,descricao_secao,quociente_localizacao
820,2007,MG,Norte de Minas,"Administração Pública, Defesa E Seguridade Social",1.92
832,2007,MG,Norte de Minas,"Agricultura, Pecuária, Produção Florestal, Pes...",0.96
844,2007,MG,Norte de Minas,Alojamento E Alimentação,0.63
856,2007,MG,Norte de Minas,"Artes, Cultura, Esporte E Recreação",0.69
868,2007,MG,Norte de Minas,Atividades Administrativas E Serviços Compleme...,0.44
...,...,...,...,...,...
976,2024,MG,Norte de Minas,Outras Atividades De Serviços,1.38
988,2024,MG,Norte de Minas,Saúde Humana E Serviços Sociais,1.16
1000,2024,MG,Norte de Minas,Serviços Domésticos,0.73
1012,2024,MG,Norte de Minas,"Transporte, Armazenagem E Correio",0.86


In [66]:
df.query('nome_mesorregiao == "Norte de Minas"')

,ano,sigla_uf,nome_mesorregiao,descricao_secao,quociente_localizacao
820,2011,MG,Norte de Minas,"Administração Pública, Defesa E Seguridade Social",1.62
832,2011,MG,Norte de Minas,"Agricultura, Pecuária, Produção Florestal, Pes...",1.06
844,2011,MG,Norte de Minas,Alojamento E Alimentação,0.68
856,2011,MG,Norte de Minas,"Artes, Cultura, Esporte E Recreação",0.82
868,2011,MG,Norte de Minas,Atividades Administrativas E Serviços Compleme...,0.52
880,2011,MG,Norte de Minas,"Atividades Financeiras, De Seguros E Serviços ...",0.56
892,2011,MG,Norte de Minas,Atividades Imobiliárias,0.43
904,2011,MG,Norte de Minas,"Atividades Profissionais, Científicas E Técnicas",0.56
916,2011,MG,Norte de Minas,Comércio; Reparação De Veículos Automotores E ...,1.21
928,2011,MG,Norte de Minas,Construção,0.61


---